In [1]:
from cng.utils import *

import ibis
from ibis import _

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
set_secrets(con)
duckdb_install_h3()

parquet = "https://minio.carlboettiger.info/public-data/social-vulnerability/2022/SVI2022_US_tract.parquet" # faster with local
con.raw_sql(f"CREATE  OR REPLACE VIEW svi AS SELECT STATE, COUNTY, FIPS, Shape AS geom FROM '{parquet}'")


In [22]:
for zoom in range(5,10):
  print(zoom)
  con.sql(f'''
  WITH t2 AS (
    WITH t1 AS (
      SELECT STATE, COUNTY, FIPS, ST_Dump(geom) AS geom 
      FROM svi
    ) 
    SELECT STATE, COUNTY, FIPS, 
          h3_polygon_wkt_to_cells_string(UNNEST(geom).geom, {zoom}) AS h{zoom}
    FROM t1
  )
  SELECT STATE, COUNTY, FIPS, UNNEST(h{zoom}) AS h{zoom} FROM t2
  ''').to_parquet(f"s3://public-social-vulnerability/2022-tracts-h3-z{zoom}.parquet")

5


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

6


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

7


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

8


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

9


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [2]:
zoom = 8
tracts = con.read_parquet(f"s3://public-social-vulnerability/2022-tracts-h3-z{zoom}.parquet")
gbif = con.read_parquet("s3://public-gbif/2024-10-01/**")

tracts.filter(_.STATE=="Vermont").join(gbif, f"h{zoom}").to_parquet("test.parquet")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))